In [1]:
import pandas as pd
import numpy as np
from gensim.test.utils import common_texts
import jieba

In [2]:
file_path = ('train_dataset_v2.tsv')
script_ids = []
scene_nums = []
sentence_nums = []
ids = []
contents = []
characters = []
emotions = []
index = 0
with open(file_path,'r',encoding='utf-8') as f:
    for line in f.readlines():
        if index > 0:
            item = line.replace('\n','').split('\t')
            id,content,character,emotion = item[0],item[1],item[2],item[3]
            script_id,scene_num,sentence_num = id.split('_')[0],id.split('_')[1],id.split('_')[3]
            script_ids.append(script_id)
            scene_nums.append(scene_num)
            sentence_nums.append(sentence_num)
            ids.append(id)
            contents.append(content)
            characters.append(character)
            emotions.append(emotion)
        index += 1
print(len(script_ids),len(scene_nums),len(sentence_nums),len(ids),len(contents),len(characters),len(emotions),index)
train = pd.DataFrame()
train['ids'] = ids
train['script_ids'] = script_ids
train['scene_nums'] = scene_nums
train['sentence_nums'] = sentence_nums
train['contents'] = contents
train['characters'] = characters
train['emotions'] = emotions
print(train.shape)
train = train[train['emotions'] != '']
print(train.shape)
train['emotions'] = train['emotions'].apply(lambda x: [int(_i) for _i in x.split(',')])
train[['love', 'joy', 'fright', 'anger', 'fear', 'sorrow']] = train['emotions'].values.tolist()
train = train.reset_index(drop=True)

42790 42790 42790 42790 42790 42790 42790 42791
(42790, 7)
(36782, 7)


In [3]:
file_path = ('test_dataset.tsv')
script_ids = []
scene_nums = []
sentence_nums = []
ids = []
contents = []
characters = []
emotions = []
index = 0
with open(file_path,'r',encoding='utf-8') as f:
    for line in f.readlines():
        if index > 0:
            item = line.replace('\n','').split('\t')
            id,content,character = item[0],item[1],item[2]
            script_id,scene_num,sentence_num = id.split('_')[0],id.split('_')[1],id.split('_')[3]
            script_ids.append(script_id)
            scene_nums.append(scene_num)
            sentence_nums.append(sentence_num)
            ids.append(id)
            contents.append(content)
            characters.append(character)
        index += 1
print(len(script_ids),len(scene_nums),len(sentence_nums),len(ids),len(contents),len(characters),index)
test = pd.DataFrame()
test['ids'] = ids
test['script_ids'] = script_ids
test['scene_nums'] = scene_nums
test['sentence_nums'] = sentence_nums
test['contents'] = contents
test['characters'] = characters
print(test.shape)
test = test.reset_index(drop=True)

21376 21376 21376 21376 21376 21376 21377
(21376, 6)


In [4]:
test.head()

,ids,script_ids,scene_nums,sentence_nums,contents,characters
0,34170_0002_A_12,34170,0002,12,穿着背心的b1醒来，看看手机，三点了。,b1
1,34170_0002_A_14,34170,0002,14,b1走出卧室。,b1
2,34170_0003_A_16,34170,0003,16,b1拿着手机，点开计时功能。,b1
3,34170_0003_A_17,34170,0003,17,b1站在淋浴头下面，水从b1的头和脸上冲刷而过。,b1
4,34170_0003_A_18,34170,0003,18,b1摈着呼吸。,b1


In [5]:
# w2v
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
strs1 = list(train['contents'].values)
strs2 = list(test['contents'].values)
strs = list(set(strs1) | set(strs2)) 
content = []
count = 0
punctuation_string = "（）():：，。！*？…、·/—．"
for str in strs:
    for i in punctuation_string:
        str = str.replace(i, '')
    seg_list = jieba.cut(str, cut_all=False)
    content.append(list(seg_list))
model = Word2Vec(sentences=content, vector_size=32, window=5, min_count=1, workers=4)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\mh_chen\AppData\Local\Temp\jieba.cache
Loading model cost 0.758 seconds.
Prefix dict has been built successfully.


In [6]:
# train
w2v = []
for idx,row in train.iterrows():
    text = row['contents']
    for i in punctuation_string:
        text = text.replace(i, '')
    seg_list = jieba.cut(text, cut_all=False)
    vector = model.wv[seg_list] 
    vector = np.mean(vector,axis=0)
    w2v.append(vector)
w2v_df = pd.DataFrame(w2v,columns=[f'w2v_{i}' for i in range(32)])
train  = pd.concat([train, w2v_df], axis=1)
print(train.shape)
# test
w2v = []
for idx,row in test.iterrows():
    text = row['contents']
    for i in punctuation_string:
        text = text.replace(i, '')
    seg_list = jieba.cut(text, cut_all=False)
    vector = model.wv[seg_list] 
    vector = np.mean(vector,axis=0)
    w2v.append(vector)
w2v_df = pd.DataFrame(w2v,columns=[f'w2v_{i}' for i in range(32)])
test  = pd.concat([test, w2v_df], axis=1)
print(test.shape)

(36782, 45)
(21376, 38)


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
content2 = [' '.join(i) for i in content]
tfidf_model = TfidfVectorizer().fit(content2)
svdT = TruncatedSVD(n_components=32,random_state=2021)
# 所有
content = []
for idx,row in train.iterrows():
    text = row['contents']
    for i in punctuation_string:
        text = text.replace(i, '')
    seg_list = jieba.cut(text, cut_all=False)
    content.append(','.join(list(seg_list)))
for idx,row in test.iterrows():
    text = row['contents']
    for i in punctuation_string:
        text = text.replace(i, '')
    seg_list = jieba.cut(text, cut_all=False)
    content.append(','.join(list(seg_list)))
print(len(content))

58158


In [8]:
# tfidf + svd
svdT.fit(tfidf_model.transform(content))
svdTFit1 = svdT.transform(tfidf_model.transform(content[0:36782]))
svdTFit2 = svdT.transform(tfidf_model.transform(content[36782:]))
svd_df = pd.DataFrame(svdTFit1,columns=[f'tfidf_{i}' for i in range(32)])
train  = pd.concat([train, svd_df], axis=1)
svd_df = pd.DataFrame(svdTFit2,columns=[f'tfidf_{i}' for i in range(32)])
test  = pd.concat([test, svd_df], axis=1)
print(train.shape)
print(test.shape)

(36782, 77)
(21376, 70)


In [9]:
train['characters'] = train['characters'].astype('category')
train['characters'] = train['characters'].cat.codes
test['characters'] = test['characters'].astype('category')
test['characters'] = test['characters'].cat.codes

In [10]:
features= [f'w2v_{i}' for i in range(32)] + [f'tfidf_{i}' for i in range(32)] + ['script_ids','scene_nums','sentence_nums']
train_data = train[features].values
test_data = test[features].values
print(train_data.shape)

(36782, 67)


In [ ]:
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
target_col = ['love', 'joy', 'fright', 'anger', 'fear', 'sorrow']

count = 0
res = []
for col in target_col:
    skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=2021)
    train_y = train[col].values
    y_pred = np.zeros((test_data.shape[0],))
    for train_index, val_index in skf.split(train_data, train_y):
        x_train, x_val = train_data[train_index], train_data[val_index]
        y_train, y_val = train_y[train_index], train_y[val_index]
        train_set = lgb.Dataset(x_train, y_train)
        val_set = lgb.Dataset(x_val, y_val)

        params = {
            "objective": "rmse",
            "metric": "rmse",
#             "num_class": 4,
            "num_leaves": 64,
            "max_depth": -1,
            "learning_rate": 0.01,
            "bagging_fraction": 0.8,  # subsample
            "feature_fraction": 0.8,  # colsample_bytree
            "bagging_freq": 5,  # subsample_freq
            "bagging_seed": 2021,
            "unbalanced":True
        }

        print('training begin')
        model = lgb.train(params, train_set, num_boost_round=5000, early_stopping_rounds=50,
                          valid_sets=[val_set], verbose_eval=200)
        y_pred += model.predict(test_data, num_iteration=model.best_iteration) / skf.n_splits
    res.append(y_pred)

training begin
[LightGBM] [Warning] Unknown parameter: unbalanced
[LightGBM] [Warning] Unknown parameter: unbalanced
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005410 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16778
[LightGBM] [Info] Number of data points in the train set: 29425, number of used features: 67
[LightGBM] [Warning] Unknown parameter: unbalanced
[LightGBM] [Info] Start training from score 0.066474
Training until validation scores don't improve for 50 rounds


C:\Users\mh_chen\anaconda3\envs\DGL\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\mh_chen\anaconda3\envs\DGL\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[200]	valid_0's rmse: 0.363538
[400]	valid_0's rmse: 0.357945
[600]	valid_0's rmse: 0.355436
[800]	valid_0's rmse: 0.353605
[1000]	valid_0's rmse: 0.352344
[1200]	valid_0's rmse: 0.351651
[1400]	valid_0's rmse: 0.351081
Early stopping, best iteration is:
[1427]	valid_0's rmse: 0.350996
training begin
[LightGBM] [Warning] Unknown parameter: unbalanced
[LightGBM] [Warning] Unknown parameter: unbalanced
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005846 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16780
[LightGBM] [Info] Number of data points in the train set: 29425, number of used features: 67
[LightGBM] [Warning] Unknown parameter: unbalanced
[LightGBM] [Info] Start training from score 0.066508
Training until validation scores don't improve for 50 rounds
[200]	valid_0's rmse: 0.36005
[400]	valid_0's rmse: 0.354447
[600]	valid_0's rmse: 0.352121
[800]	valid_0's rmse: 0.350809
[1000]	valid_0'

C:\Users\mh_chen\anaconda3\envs\DGL\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\mh_chen\anaconda3\envs\DGL\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[200]	valid_0's rmse: 0.375709
[400]	valid_0's rmse: 0.370821
[600]	valid_0's rmse: 0.368396
[800]	valid_0's rmse: 0.367253
[1000]	valid_0's rmse: 0.366458
[1200]	valid_0's rmse: 0.365807
[1400]	valid_0's rmse: 0.365382
Early stopping, best iteration is:
[1372]	valid_0's rmse: 0.365344
training begin
[LightGBM] [Warning] Unknown parameter: unbalanced
[LightGBM] [Warning] Unknown parameter: unbalanced
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009720 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16779
[LightGBM] [Info] Number of data points in the train set: 29425, number of used features: 67
[LightGBM] [Warning] Unknown parameter: unbalanced
[LightGBM] [Info] Start training from score 0.100833
Training until validation scores don't improve for 50 rounds
[200]	valid_0's rmse: 0.37561
[400]	valid_0's rmse: 0.369555
[600]	valid_0's rmse: 0.367344
[800]	valid_0's rmse: 0.36646
Early stopping, 

...

[LightGBM] [Info] Start training from score 0.166525
Training until validation scores don't improve for 50 rounds
[200]	valid_0's rmse: 0.521306
[400]	valid_0's rmse: 0.513485
[600]	valid_0's rmse: 0.511467
[800]	valid_0's rmse: 0.510067
[1000]	valid_0's rmse: 0.509087
Early stopping, best iteration is:
[1087]	valid_0's rmse: 0.509028
training begin
[LightGBM] [Warning] Unknown parameter: unbalanced
[LightGBM] [Warning] Unknown parameter: unbalanced
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005946 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16779
[LightGBM] [Info] Number of data points in the train set: 29425, number of used features: 67
[LightGBM] [Warning] Unknown parameter: unbalanced
[LightGBM] [Info] Start training from score 0.166525
Training until validation scores don't improve for 50 rounds
[200]	valid_0's rmse: 0.522773
[400]	valid_0's rmse: 0.513628
[600]	valid_0's rmse: 0.50

C:\Users\mh_chen\anaconda3\envs\DGL\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\mh_chen\anaconda3\envs\DGL\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[200]	valid_0's rmse: 0.444324
[400]	valid_0's rmse: 0.436476
[600]	valid_0's rmse: 0.43454
Early stopping, best iteration is:
[669]	valid_0's rmse: 0.43403
training begin
[LightGBM] [Warning] Unknown parameter: unbalanced
[LightGBM] [Warning] Unknown parameter: unbalanced
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013312 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16779
[LightGBM] [Info] Number of data points in the train set: 29425, number of used features: 67
[LightGBM] [Warning] Unknown parameter: unbalanced
[LightGBM] [Info] Start training from score 0.130161
Training until validation scores don't improve for 50 rounds
[200]	valid_0's rmse: 0.452961
[400]	valid_0's rmse: 0.446044
[600]	valid_0's rmse: 0.444585
[800]	valid_0's rmse: 0.443763
Early stopping, best iteration is:
[861]	valid_0's rmse: 0.44371
training begin
[LightGBM] [Warning] Unknown parameter: unbalanced
[LightGBM] [W

C:\Users\mh_chen\anaconda3\envs\DGL\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\mh_chen\anaconda3\envs\DGL\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[200]	valid_0's rmse: 0.627289
[400]	valid_0's rmse: 0.61764
[600]	valid_0's rmse: 0.613521
[800]	valid_0's rmse: 0.611556
[1000]	valid_0's rmse: 0.610213
Early stopping, best iteration is:
[1072]	valid_0's rmse: 0.610043
training begin
[LightGBM] [Warning] Unknown parameter: unbalanced
[LightGBM] [Warning] Unknown parameter: unbalanced
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005634 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16779
[LightGBM] [Info] Number of data points in the train set: 29425, number of used features: 67
[LightGBM] [Warning] Unknown parameter: unbalanced
[LightGBM] [Info] Start training from score 0.263925
Training until validation scores don't improve for 50 rounds
[200]	valid_0's rmse: 0.628182
[400]	valid_0's rmse: 0.617605
[600]	valid_0's rmse: 0.613342
[800]	valid_0's rmse: 0.611166
[1000]	valid_0's rmse: 0.610264
Early stopping, best iteration is:
[1074]	valid

In [ ]:
submit = pd.read_csv('submit_example.tsv', sep='\t')

In [ ]:
for idx,row in submit.iterrows():
    row['emotion'] = f'{round(res[0][idx])}' + ',' + f'{round(res[1][idx])}'+ ',' + f'{round(res[2][idx])}'+ ',' + f'{round(res[3][idx])}'+ ',' + f'{round(res[4][idx])}'+ ',' + f'{round(res[5][idx])}'

In [ ]:
submit

In [ ]:
submit.to_csv('submit.txt',sep='\t',index=None)